In [12]:
## NPI Reader - 10, May 24, 2024
## To specify the input file and which years to extract, set the variables defined near line 900

## Kicks out on any of 3 errors found: pecos, no claims data, unsupported year
#
# In the function that calls fetch_info, have it catch if the unsupported year occurred and if it did, have it
# try calling the function 5 more times

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import csv
import time

import json
import requests
from bs4 import BeautifulSoup
import re
import os
import hashlib

def fetch_info(npi, year):
    #print("Input npi to fetch_info() = ", npi)
    #url = f"https://qpp.cms.gov/participation-lookup?npi={npi}&py=2024"
    #url = f"https://qpp.cms.gov/participation-lookup?npi={npi}&py=2023"
    print("Year: ", year)
    url = f"https://qpp.cms.gov/participation-lookup?npi={npi}&py={year}"
   
    # Initialize all the variable fields that we'll be attempting to parse. If the field can't be parsed, then "temp" will be returned and the function won't error out
    npi_value = "temp"
    associated_practices = "temp"
    physician_name = "temp"
    practice_name = "temp"
    practice_address = "temp"
    mips_info = "temp"
    individual_status = "temp"
    group_status = "temp"
    optin_text = "temp"
    mips_reporting_requirements = "temp"
    mips_reporting_options = "temp"
    payment_information = "temp"
    cl_exceeds_low_volume = "temp" 
    cl_med_patients = "temp"
    cl_allowed_charges = "temp"
    cl_covered_services = "temp"
    cl_mips_ec_type = "temp"
    cl_enroll_b4_2024 = ""
    pl_exceed_lv = "temp"
    pl_medicare_pts = "temp"
    pl_allowed_charges = "temp"
    apm_participation = "temp" 
    apm_participation_num = "temp"
    apm_participation_text = "temp"
    apm_participation_comment = "temp"
    apm_name = "temp"
    apm_details_classification = ""   # if APM Details (Classification) doesn't exist, put an empty string in the csv file
    apm_details_model = ""   # if APM Details (Model) doesn't exist, put an empty string in the csv file
    apm_details_participation_details = ""   # if APM Details (Participation Details) doesn't exist, put an empty string in the csv file
    data_fetch_status = "Success"
    cl_hospital_based_status = "No" 
    cl_npfacing_status = "No" 
    cl_rural_status = "No" 
    cl_small_practice_status = "No" 
    pl_hospital_based_status = "No" 
    pl_rural_status = "No" 
    pl_small_practice_status = "No" 
    cl_hspa_status = "No"
    pl_hpsa_status = "No"
    cl_hardship = "No"  
    

    

    try:
        #print(url)
        driver.get(url)
        

        # Function to get the hash of the page source
        def get_page_hash():
            return hashlib.md5(driver.page_source.encode('utf-8')).hexdigest()   
        # Initialize old and new hashes
        old_page_hash = None
        new_page_hash = get_page_hash()   
        # Initialize start time for the loop
        start_time = time.time()
        timeout = 30  # seconds    
        # Loop until the page source is the same between checks or timeout occurs
        while time.time() - start_time < timeout:
            if old_page_hash == new_page_hash:
                print("Page has stabilized.")
                break
            else:
                print("Page has NOT stabilized.")
                time.sleep(1)  # Wait for 2 seconds before rechecking
                old_page_hash = new_page_hash
                new_page_hash = get_page_hash()
    
        # Check if the loop exited due to a timeout
        if old_page_hash != new_page_hash:
            print("Page did not stabilize within the timeout period.")
            raise TimeoutException("Page did not stabilize within the timeout period.")

        
        # Use this section to output the complete generated html file
        #print("############### START OF HTMLDUMP ##############################")
        #print()
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        #print(html) # the preceding line captured the entire dynamically generated html
        #print()
        #print("############### END OF HTMLDUMP ##############################")
        #print()

    
        # mike - let's see if we get to here!
        #print(driver)
       
       # Parse the HTML content of the page
        soup = BeautifulSoup(html, 'html.parser')     
        
        # Look for all instances of <div class="qpp-c-alert__body">
        error_divs = soup.find_all('div', class_="qpp-c-alert__body")
        expected_error_message = "The National Provider Identifier (NPI) does not have claims data for this given period."
        expected_error_message_pecos = "The National Provider Identifier (NPI) does not exist in the PECOS import yet. Please try again later."
        expected_error_message_not_supported = "is not a supported performance year"

        
        # Iterate over each found error div and check if it contains the expected text
        for error_div in error_divs:
            if expected_error_message in error_div.get_text(strip=True):
                print("Specific error message found:", error_div.get_text(strip=True))
                return [npi, error_div.get_text(strip=True), year]
            if expected_error_message_pecos in error_div.get_text(strip=True):
                print("Specific error message found:", error_div.get_text(strip=True))
                return [npi, error_div.get_text(strip=True), year]
            if expected_error_message_not_supported in error_div.get_text(strip=True):
                print("Specific error message found:", error_div.get_text(strip=True))
                data_fetch_status = "NPI section not found."
                #print(html)
                # Check if the errors folder exists, create it if not
                error_dir = 'errors'
                if not os.path.exists(error_dir):
                    os.makedirs(error_dir)

                ## When npi_div is not found, save the HTML to a file within the errors subfolder
                filename = os.path.join(error_dir, f"{npi}.html")
                print("filename: ", filename)
                with open(filename, 'w', encoding='utf-8') as file:
                    file.write(html)  # Write the HTML content to the file
                print(f"HTML content has been written to {filename} for further inspection.")                
                return [npi, error_div.get_text(strip=True), year]
    


        
        # Locate the <div> with class 'npi'
        npi_div = soup.find('div', class_='npi')
        #print("npi_div =", npi_div)
        if npi_div:
            npi_value = npi_div.get_text(strip=True)  # Extract the text content, stripping whitespace
            #print("NPI: ", npi_value)
        else:
            print("NPI section not found.")
            data_fetch_status = "NPI section not found."
            #print(html)
            # Check if the errors folder exists, create it if not
            error_dir = 'errors'
            if not os.path.exists(error_dir):
                os.makedirs(error_dir)

            ## When npi_div is not found, save the HTML to a file within the errors subfolder
            filename = os.path.join(error_dir, f"{npi}.html")
            print("filename: ", filename)
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(html)  # Write the HTML content to the file
            print(f"HTML content has been written to {filename} for further inspection.")
            return [npi, data_fetch_status, year]  
            
       ## extract Associated Practices
       # Find the section with class 'practice-details'
        section = soup.find('section', class_='practice-details')

        # Initialize a variable to store the extracted number
        number_of_practices = None

        # Find the h4 tag and extract the number
        if section:
            h4_text = section.find('h4').get_text()  # Get the text of the h4 tag
            # Extract the number from the string
            start = h4_text.find('(') + 1
            end = h4_text.find(')')
            if start > 0 and end > 0:
                associated_practices = int(h4_text[start:end])  # Convert the substring to an integer
        print("Number of Associated Practices:", associated_practices)
            
            
            
        ## Extract physician name, practice name, and practice address    
        practice_header = soup.find('div', class_='practice-header')
        if practice_header:
            # Extracting the physician's name, practice name, and practice address
            physician_name_untruncated = practice_header.find('h5').get_text(strip=True)
            physician_name = physician_name_untruncated.split(" at")[0]   #The html is of the form "Phys name at Practice Name"
            practice_name = practice_header.find('span').next_sibling.strip()
            practice_address = practice_header.find('div', class_='address').get_text(strip=True)
        
            #print("MIPS Individual: ", eligibility_info )
 
            #print("Physician Name: ", physician_name)
            #print("Practice  Name: ",practice_name)
            #print("Practice Address: ",practice_address)        
        else:
            data_fetch_status = "Practice header section not found"
            return [npi, data_fetch_status]  
        
        
        ## Extract whether Whether a physician is classified as “MIPS Eligible Individual” or “MIPS Exempt Individual” and whether a physician is classified as “MIPS Eligible Group” or “MIPS Exempt Group"
        reporting_requirements_div = soup.find("div", class_="reporting-requirements")
        # If the specific div is not found, return an error or unknown status
        if not reporting_requirements_div:
            return {"Individual Status": "Unknown", "Group Status": "Unknown"}
    
        individual_span = reporting_requirements_div.find("span", {"aria-label": lambda value: value and "Individual" in value})
        group_span = reporting_requirements_div.find("span", {"aria-label": lambda value: value and "Group" in value})
    
        individual_status = individual_span['aria-label'] if individual_span else "Unknown"
        group_status = group_span['aria-label'] if group_span else "Unknown"
        #print("individual_status: ", individual_status)  
        #print("group_status: ", group_status)       
        
        
        ## Extract the Opt In Option
        opt_in_div = soup.find("div", class_="opt-in-flag") 
        if opt_in_div:
            optin_text = opt_in_div.get_text(strip=True)
            prefix = "Opt-in Option:"
            if optin_text.startswith(prefix):
                optin_text = optin_text[len(prefix):]  # Remove the prefix from the start
        else:
            optin_text = "None"
        #print("optin_text:", optin_text)
        
        ## Scrape contents of 'MIPS Reporting Requirements'
        # Locate all 'div' elements with class 'reporting-table-label' and check their text
        table_labels = soup.find_all("div", class_="reporting-table-label")
        for label in table_labels:
            if label.get_text(strip=True) == "MIPS Reporting Requirements":
                # Get the next sibling 'div' element which is 'reporting-table-field'
                content_div = label.find_next_sibling("div", class_="reporting-table-field")
                if content_div:
                    inner_content = content_div.find("div", class_="reporting-inner-content")
                    mips_reporting_requirements = inner_content.get_text(strip=True) if inner_content else "Content not found"
                    break  # Stop searching once the correct section is found
        #print("mips_reporting_requirements:", mips_reporting_requirements)

        ## Scrape contents of 'MIPS REPORTING & PARTICIPATION OPTIONS'
        # Locate all 'div' elements with class 'reporting-table-label' and check their text
        table_labels = soup.find_all("div", class_="reporting-table-label")
        for label in table_labels:
            if label.get_text(strip=True) == "MIPS Reporting & Participation Options":
                # Get the next sibling 'div' element which is 'reporting-table-field'
                content_div = label.find_next_sibling("div", class_="reporting-table-field")
                if content_div:
                    inner_content = content_div.find("div", class_="reporting-inner-content")
                    mips_reporting_options = inner_content.get_text(strip=True) if inner_content else "Content not found"
                    break  # Stop searching once the correct section is found
        #print("mips_reporting_options:", mips_reporting_options)
       
        ## Scrape contents of 'Payment Information'
        # Locate all 'div' elements with class 'reporting-table-label' and check their text
        table_labels = soup.find_all("div", class_="reporting-table-label")
        for label in table_labels:
            if label.get_text(strip=True) == "Payment Information":
                # Get the next sibling 'div' element which is 'reporting-table-field'
                content_div = label.find_next_sibling("div", class_="reporting-table-field")
                if content_div:
                    inner_content = content_div.find("div", class_="reporting-inner-content")
                    payment_information = inner_content.get_text(strip=True) if inner_content else "Content not found"
                    break  # Stop searching once the correct section is found
        #print("payment_information:", payment_information)

        
        ## Scrape contents for (Clinician Level) Exceeds low volume threshold
        #cl_exceeds_low_volume = "Information not found"  # Initialize the variable to hold extracted information
        # First locate the 'div' with class 'scenario'
        scenario_div = soup.find("div", class_="scenario")
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "Exceeds low volume threshold" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_exceeds_low_volume = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_exceeds_low_volume = "Clinician level information section not found"
        else:
            cl_exceeds_low_volume = "Scenario section not found"
        #print("cl_exceeds_low_volume: ", cl_exceeds_low_volume)

        
        ## Scrape contents for (Clinician Level) Medicare patients for this clinician
        # First locate the 'div' with class 'scenario'
        scenario_div = soup.find("div", class_="scenario")
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "Medicare patients for this clinician" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_med_patients = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_med_patients = "Clinician level information section not found"
        else:
            cl_med_patients = "Scenario section not found"
        #print("cl_med_patients: ", cl_med_patients)

        
        ## Scrape contents for (Clinician Level) Allowed charges for this clinician
        scenario_div = soup.find("div", class_="scenario")
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "Allowed charges for this clinician" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_allowed_charges = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_allowed_charges = "Clinician level information section not found"
        else:
            cl_allowed_charges = "Scenario section not found"
        #print("cl_allowed_charges: ", cl_allowed_charges)
        
        
        ## Scrape contents for (Clinician Level) Covered services for this clinician
        #scenario_div = soup.find("div", class_="scenario")  # WE GOT THIS IN THE PRECEDING SECTION, NO NEED TO RE_FETCH
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "Covered services for this clinician" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_covered_services = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_covered_services = "Clinician level information section not found"
        else:
            cl_covered_services = "Scenario section not found"
        #print("cl_covered_services: ", cl_covered_services)

        
        ##Scrape contents for (Clinician Level) MIPS eligible clinician type
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "MIPS eligible clinician type" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_mips_ec_type = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_mips_ec_type = "Clinician level information section not found"
        else:
            cl_mips_ec_type = "Scenario section not found"
        #print("cl_mips_ec_type: ", cl_mips_ec_type)
 

        ##Scrape contents for (Clinician Level) Enrolled in Medicare before January 1, 2024
        if scenario_div:
            # Within 'scenario', locate the specific 'div' for Clinician Level Information
            clinician_level_info = scenario_div.find("div", class_="h8", text="Clinician Level Information")
            if clinician_level_info:
                # Locate all 'div' elements with class 'detail' following the specific 'div'
                detail_blocks = clinician_level_info.find_next_sibling("div", class_="detail-boxed").find_all("div", class_="detail")
                for block in detail_blocks:
                    # Check if the label matches "Exceeds low volume threshold"
                    label = block.find("div", class_="detail-label")
                    if label and "Enrolled in Medicare before January 1, 2024" in label.get_text(strip=True):
                        field = block.find("div", class_="detail-field")
                        cl_enroll_b4_2024 = field.get_text(strip=True) if field else "Field not found"
                        break  # Stop searching once the correct section is found
            else:
                cl_enroll_b4_2024 = "Clinician level information section not found"
        else:
            cl_enroll_b4_2024 = "Scenario section not found"
        #print("cl_enroll_b4_2024: ", cl_enroll_b4_2024)

        ## Scrape contents for (Practice Level) Exceeds low volume threshold
        # Directly locate the 'h8' div with the specified text, and then navigate to sibling details
        practice_level_info = soup.find("div", class_="h8", text="Practice Level Information")
        if practice_level_info:
            detail_boxed = practice_level_info.find_next_sibling("div", class_="detail-boxed")
            if detail_boxed:
                details = detail_boxed.find_all("div", class_="detail")
                for detail in details:
                    label = detail.find("div", class_="detail-label")
                    if label and "Exceeds low volume threshold" in label.get_text(strip=True):
                        field = detail.find("div", class_="detail-field")
                        pl_exceed_lv = field.get_text(strip=True) if field else "Field not found"
                        break  # Found the correct label and extracted the field text
            else:
                pl_exceed_lv = "Detail boxed section not found"
        else:
            pl_exceed_lv = "Practice level information section not found"
        #print("pl_exceed_lv: ", pl_exceed_lv)

        ## Scrape contents for (Practice Level) Medicare patients at this practice
        # Directly locate the 'h8' div with the specified text, and then navigate to sibling details
        #practice_level_info = soup.find("div", class_="h8", text="Practice Level Information")   #don't need to find this again since it was found in the previous block
        if practice_level_info:
            detail_boxed = practice_level_info.find_next_sibling("div", class_="detail-boxed")
            if detail_boxed:
                details = detail_boxed.find_all("div", class_="detail")
                for detail in details:
                    label = detail.find("div", class_="detail-label")
                    if label and "Medicare patients at this practice" in label.get_text(strip=True):
                        field = detail.find("div", class_="detail-field")
                        pl_medicare_pts = field.get_text(strip=True) if field else "Field not found"
                        break  # Found the correct label and extracted the field text
            else:
                pl_medicare_pts = "Detail boxed section not found"
        else:
            pl_medicare_pts = "Practice level information section not found"
        #print("pl_medicare_pts: ", pl_medicare_pts)
        

        ## Scrape contents for (Practice Level) Allowed charges at this practice
        # Directly locate the 'h8' div with the specified text, and then navigate to sibling details
        #practice_level_info = soup.find("div", class_="h8", text="Practice Level Information")   #don't need to find this again since it was found in the previous block
        if practice_level_info:
            detail_boxed = practice_level_info.find_next_sibling("div", class_="detail-boxed")
            if detail_boxed:
                details = detail_boxed.find_all("div", class_="detail")
                for detail in details:
                    label = detail.find("div", class_="detail-label")
                    if label and "Allowed charges at this practice" in label.get_text(strip=True):
                        field = detail.find("div", class_="detail-field")
                        pl_allowed_charges = field.get_text(strip=True) if field else "Field not found"
                        break  # Found the correct label and extracted the field text
            else:
                pl_allowed_charges = "Detail boxed section not found"
        else:
            pl_allowed_charges = "Practice level information section not found"
        #print("pl_allowed_charges: ", pl_allowed_charges)

        
        ## Scrape contents for (Practice Level) Covered services at this practice
        # Directly locate the 'h8' div with the specified text, and then navigate to sibling details
        #practice_level_info = soup.find("div", class_="h8", text="Practice Level Information")   #don't need to find this again since it was found in the previous block
        if practice_level_info:
            detail_boxed = practice_level_info.find_next_sibling("div", class_="detail-boxed")
            if detail_boxed:
                details = detail_boxed.find_all("div", class_="detail")
                for detail in details:
                    label = detail.find("div", class_="detail-label")
                    if label and "Covered services at this practice" in label.get_text(strip=True):
                        field = detail.find("div", class_="detail-field")
                        pl_covered_services = field.get_text(strip=True) if field else "Field not found"
                        break  # Found the correct label and extracted the field text
            else:
                pl_covered_services = "Detail boxed section not found"
        else:
            pl_covered_services = "Practice level information section not found"
        #print("pl_covered_services: ", pl_covered_services)
        
        ## Extract APM Participation (x) info
        # Locate the <h6> tag containing the string "APM Participation"
        h6_tag = soup.find("h6", text=lambda text: text and "APM Participation" in text)
        if h6_tag:
            apm_participation = h6_tag.get_text(strip=True)
        else:
            apm_participation = "No <h6> tag with 'APM Participation' found"
        #print("apm_participation: ", apm_participation)
 
        ## Extract APM Participation (x) info
        # Locate the <h6> tag containing the string "APM Participation"
        #h6_tag = soup.find("h6", text=lambda text: text and "APM Participation" in text)
        if h6_tag:
            # Extract only the number inside the parentheses using regex
            match = re.search(r'\((\d+)\)', h6_tag.get_text())
            if match:
                apm_participation_num = match.group(1)  # Group 1 is the first captured group, the number inside the parentheses
            else:
                apm_participation_num = "No number found in <h6> tag"
        else:
            apm_participation_num = "No <h6> tag with 'APM Participation' found"   
        #print("apm_participation_num: ", apm_participation_num)
   
 
        ## Extract "Check APM Requirements" or "NOT REQUIRED TO REPORT FOR ANY APMS"
        # we're using the same h6_tag from the previous block, so no need to refind it
        #h6_tag = soup.find("h6", text=lambda text: text and "APM Participation" in text)
        if h6_tag:
            # Find the next sibling span with the specific class
            span_tag = h6_tag.find_next_sibling("span", class_="reporting-requirements no-label")
            if span_tag:
                apm_participation_text = span_tag.get_text(strip=True)
            else:
                apm_participation_text = "No following span with class 'reporting-requirements no-label' found"
        else:
            apm_participation_text = "No <h6> tag with 'APM Participation' found"
        #print("apm_participation_text: ", apm_participation_text)
 
    
       ## Extract APM Participation Requirements Comment
       # Find the <h6> with 'APM Participation' and limit searches to its parent <section>
        h6_tag = soup.find("h6", text=lambda text: "APM Participation" in text)
        if h6_tag:
            section_tag = h6_tag.parent  # Assuming the <h6> is directly under the <section>
            span_tag = section_tag.find("span", class_="reporting-requirements no-label")
            if span_tag:
                p_tag = span_tag.find_next("p")  # Find the next p tag after the span
                if p_tag:
                    #apm_participation_comment = p_tag.get_text(strip=True) if p_tag.text else "<p> tag is empty"
                    apm_participation_comment = p_tag.get_text(strip=True) if p_tag.text else "" #return an empty string so that it looks better in the csv file                  
                else:
                    apm_participation_comment = "No <p> tag found after the span"
            else:
                apm_participation_comment = "No <span> tag with class 'reporting-requirements no-label' found"
        else:
            apm_participation_comment = "No <h6> tag with 'APM Participation' found"
        #print("apm_participation_comment: ", apm_participation_comment)

        ## Extract the APN Name (if it exists)
        # Find the <h6> with 'APM Participation' and ensure it's the correct section
        h6_tag = soup.find("h6", text=lambda text: "APM Participation" in text)
        if h6_tag:
            section_tag = h6_tag.find_parent('section')  # Get the parent <section> of the <h6>
            h7_tag = section_tag.find("div", class_="h7")  # Find the <div class="h7"> within the section
            if h7_tag:
                apm_name = h7_tag.get_text(strip=True)  # Assign the text from <div class="h7">
            else:
                #apm_name = "No <div class='h7'> found within the section"
                apm_name = "" #return an empty string so it looks better in the csv file
                
        else:
            apm_name = "No <h6> with 'APM Participation' found"
        #print("apm_name: ", apm_name)


        ## Extract (APM Details) Classification
        # Find the <h6> with 'APM Participation' and ensure it's the correct section
        h6_tag = soup.find("h6", text=lambda text: "APM Participation" in text)
        if h6_tag:
            section_tag = h6_tag.find_parent('section')  # Get the parent <section> of the <h6>
            details = section_tag.find_all("div", class_="detail")
            for detail in details:
                label = detail.find("div", class_="detail-label")
                if label and label.get_text(strip=True) == "Classification":
                    detail_field = detail.find("div", class_="detail-field")
                    if detail_field:
                        apm_details_classification = detail_field.get_text(strip=True)  # Assign the text from <div class="detail-field">
                    break  # Stop after finding the first matching detail field
        else:
            apm_details_classification = "No <h6> with 'APM Participation' found"
        #print("apm_details_classification: ", apm_details_classification)

        ## Extract (APM Details) Model
        # don't extract the h6_tag again. Let's see if we can use the same one as the precding block
        if h6_tag:
            section_tag = h6_tag.find_parent('section')  # Get the parent <section> of the <h6>
            details = section_tag.find_all("div", class_="detail")
            for detail in details:
                label = detail.find("div", class_="detail-label")
                if label and label.get_text(strip=True) == "Model":
                    detail_field = detail.find("div", class_="detail-field")
                    if detail_field:
                        apm_details_model = detail_field.get_text(strip=True)  # Assign the text from <div class="detail-field">
                    break  # Stop after finding the first matching detail field
        else:
            apm_details_model = "No <h6> with 'APM Participation' found"
        #print("apm_details_model: ", apm_details_model)

        ## Extract (APM Details) Participation Details
        # don't extract the h6_tag again. Let's see if we can use the same one as the precding block
        if h6_tag:
            section_tag = h6_tag.find_parent('section')  # Get the parent <section> of the <h6>
            details = section_tag.find_all("div", class_="detail")
            for detail in details:
                label = detail.find("div", class_="detail-label")
                if label and label.get_text(strip=True) == "Participation Details":
                    detail_field = detail.find("div", class_="detail-field")
                    if detail_field:
                        apm_details_participation_details = detail_field.get_text(strip=True)  # Assign the text from <div class="detail-field">
                    break  # Stop after finding the first matching detail field
        else:
            apm_details_participation_details = "No <h6> with 'APM Participation' found"
        #print("apm_details_participation_details: ", apm_details_participation_details)

 
        #### Extract (Clinician Level) SPECIAL STATUS Hospital-based
         # Find the section 'Other Reporting Factors'
        section_header = soup.find('h6', string="Other Reporting Factors")

        if section_header:
            # Find the div that contains the 'Clinician Level' text, safely check for the div existence
            scenario_div = section_header.find_next('div', class_="scenario")
            if scenario_div:
                clinician_div = scenario_div.find('div', string="Clinician Level")
                if clinician_div and clinician_div.parent:
                    # Find the span containing 'Hospital-based' and its corresponding 'div class="detail-field"'
                    hospital_based_span = clinician_div.parent.find('span', string="Hospital-based")
                    if hospital_based_span:
                        detail_field = hospital_based_span.find_next('div', class_="detail-field")
                        if detail_field:
                            cl_hospital_based_status = detail_field.text.strip()
                    else:
                        cl_hospital_based_status = "No"
                else:
                    cl_hospital_based_status = "No"
            else:
                cl_hospital_based_status = "No"
        else:
            cl_hospital_based_status = "No"

        #print("cl_hospital_based_status:", cl_hospital_based_status)
                
        #### Extract (Clinician Level) SPECIAL STATUS Non-patient facing
        # Find the section 'Other Reporting Factors'
        section_header = soup.find('h6', string="Other Reporting Factors")

        if section_header:
            # Safely find the div that contains the 'Clinician Level' text
            scenario_div = section_header.find_next('div', class_="scenario")
            clinician_div = scenario_div.find('div', string="Clinician Level") if scenario_div else None
    
            if clinician_div and clinician_div.parent:
                # Find the span containing 'Non-patient facing' and its corresponding 'div class="detail-field"'
                npfacing_span = clinician_div.parent.find('span', string="Non-patient facing")
                if npfacing_span:
                    detail_field = npfacing_span.find_next('div', class_="detail-field")
                    if detail_field:
                        cl_npfacing_status = detail_field.text.strip()
                else:
                    cl_npfacing_status = "No"
            else:
                cl_npfacing_status = "No"
        else:
            cl_npfacing_status = "No"
        #print("cl_npfacing_status:", cl_npfacing_status)
        
        
        ### Extract (Clinician Level) SPECIAL STATUS Rural
        # Find the section 'Other Reporting Factors'
        section_header = soup.find('h6', string="Other Reporting Factors")
        if section_header:
            # Safely find the div that contains the 'Clinician Level' text
            clinician_level_div = section_header.find_next('div', class_="scenario")
            clinician_div = clinician_level_div.find('div', string="Clinician Level") if clinician_level_div else None
    
            if clinician_div and clinician_div.parent:
                # Find the span containing 'Rural' and its corresponding 'div class="detail-field"'
                rural_span = clinician_div.parent.find('span', string="Rural")
                if rural_span:
                    detail_field = rural_span.find_next('div', class_="detail-field")
                    if detail_field:
                        cl_rural_status = detail_field.text.strip()
                else:
                    cl_rural_status = "No"
            else:
                cl_rural_status = "No"
        else:
            cl_rural_status = "Section 'Other Reporting Factors' not found."
        #print("cl_rural_status:", cl_rural_status)
             
        
        ### Extract (Clinician Level) SPECIAL STATUS Small practice
        # Find the section 'Other Reporting Factors'
        section_header = soup.find('h6', string="Other Reporting Factors")
        if section_header:
            # Find the div that contains the 'Clinician Level' text
            scenario_div = section_header.find_next('div', class_="scenario")
            if scenario_div:
                clinician_div = scenario_div.find('div', string="Clinician Level")
                if clinician_div and clinician_div.parent:
                    # Find the span containing 'Small practice'
                    small_practice_span = clinician_div.parent.find('span', string="Small practice")
                    if small_practice_span:
                        detail_field = small_practice_span.find_next('div', class_="detail-field")
                        if detail_field:
                            cl_small_practice_status = detail_field.text.strip()
                    else:
                        cl_small_practice_status = "No"
                else:
                    cl_small_practice_status = "No"
            else:
                cl_small_practice_status = "No"
        else:
            cl_small_practice_status = "Section 'Other Reporting Factors' not found."
        #print("cl_small_practice_status:", cl_small_practice_status)

       ### Extract (Clinician Level) SPECIAL STATUS Health Professional Shortage Area (HPSA)
       # Find the section 'Other Reporting Factors'
        section_header = soup.find('h6', string="Other Reporting Factors")
        if section_header:
            # Find the div that contains the 'Clinician Level' text
            scenario_div = section_header.find_next('div', class_="scenario")
            if scenario_div:
                clinician_div = scenario_div.find('div', string="Clinician Level")
                if clinician_div and clinician_div.parent:
                    # Find the span containing 'Small practice'
                    small_practice_span = clinician_div.parent.find('span', string="Health Professional Shortage Area (HPSA)")
                    if small_practice_span:
                        detail_field = small_practice_span.find_next('div', class_="detail-field")
                        if detail_field:
                            cl_hspa_status = detail_field.text.strip()
                    else:
                        cl_hspa_status = "No"
                else:
                    cl_hspa_status = "No"
            else:
                cl_hspa_status = "No"
        else:
            cl_hspa_status = "Section 'Other Reporting Factors' not found."
        #print("cl_hspa_status:", cl_hspa_status)

        
       ### Extract (Clinician Level) HARDSHIP EXCEPTION Extreme and uncontrollable circumstances
        try:
            # Find the section header to ensure we are working within the correct part of the document
            section_header = soup.find('h6', string="Other Reporting Factors")
            if section_header:
                # Navigate to the 'scenario' div, checking existence at each step
                scenario_div = section_header.find_next_sibling('div', class_="scenario")
                if scenario_div:
                    # Find the 'detail-boxed' div within 'scenario'
                    detail_boxed = scenario_div.find('div', class_="detail-boxed")
                    if detail_boxed:
                        # Locate the specific 'detail' div that contains the 'Hardship Exception' information
                        detail_div = detail_boxed.find('div', class_="detail")
                        if detail_div:
                            # Finally, find the 'detail-label' div and extract the text
                            detail_label = detail_div.find('div', class_="detail-label")
                            if detail_label:
                                cl_hardship = detail_label.get_text(strip=True)
                                # Check if the extracted text matches the expected value
                                if cl_hardship == "Hardship ExceptionExtreme and uncontrollable circumstances":
                                    cl_hardship = "Extreme and uncontrollable circumstances"
                                elif cl_hardship != "Hardship ExceptionExtreme and uncontrollable circumstances":
                                    cl_hardship = "No"  # Reset to empty string if not matching
        except Exception as e:
            print(f"An error occurred: {e}")
            # cl_hardship remains an empty string if any exceptions are caught
        #print("cl_hardship:", cl_hardship)

        
        
        
        ### Extract (Practice Level) SPECIAL STATUS Hospital-based       
        # Attempt to find the section with 'Other Reporting Factors'
        section = soup.find('h6', string="Other Reporting Factors").parent if soup.find('h6', string="Other Reporting Factors") else None       
        if section:
            # Attempt to find the 'Practice Level' div
            practice_level_div = section.find('div', class_="h7", string="Practice Level")
            if practice_level_div:
                # Attempt to find the 'Hospital-based' span
                hospital_based_span = practice_level_div.find_next('span', string="Hospital-based")
                if hospital_based_span:
                    # Attempt to find the corresponding 'div class="detail-field"'
                    detail_field = hospital_based_span.find_next('div', class_="detail-field")
                    if detail_field:
                        pl_hospital_based_status = detail_field.text.strip()
                    else:
                        pass
                        #pl_hospital_based_status = "Detail field not found for 'Hospital-based'."
                else:
                    pass
                    #pl_hospital_based_status = "'Hospital-based' span not found."
            else:
                pass
                #pl_hospital_based_status = "'Practice Level' div not found."
        else:
            pl_hospital_based_status = "'Other Reporting Factors' section not found."
        #print("pl_hospital_based_status:", pl_hospital_based_status)

        
        ### Extract (Practice Level) SPECIAL STATUS Rural       
        # Attempt to find the section with 'Other Reporting Factors'
        section = soup.find('h6', string="Other Reporting Factors").parent if soup.find('h6', string="Other Reporting Factors") else None       
        if section:
            # Attempt to find the 'Practice Level' div
            practice_level_div = section.find('div', class_="h7", string="Practice Level")
            if practice_level_div:
                # Attempt to find the 'Hospital-based' span
                hospital_based_span = practice_level_div.find_next('span', string="Rural")
                if hospital_based_span:
                    # Attempt to find the corresponding 'div class="detail-field"'
                    detail_field = hospital_based_span.find_next('div', class_="detail-field")
                    if detail_field:
                        pl_rural_status = detail_field.text.strip()
                    else:
                        pass
                        #pl_hospital_based_status = "Detail field not found for 'Hospital-based'."
                else:
                    pass
                    #pl_hospital_based_status = "'Hospital-based' span not found."
            else:
                pass
                #pl_hospital_based_status = "'Practice Level' div not found."
        else:
            pl_rural_status = "'Other Reporting Factors' section not found."
        #print("pl_rural_status:", pl_rural_status)
        
        ### Extract (Practice Level) SPECIAL STATUS Small practice       
        # Attempt to find the section with 'Other Reporting Factors'
        section = soup.find('h6', string="Other Reporting Factors").parent if soup.find('h6', string="Other Reporting Factors") else None       
        if section:
            # Attempt to find the 'Practice Level' div
            practice_level_div = section.find('div', class_="h7", string="Practice Level")
            if practice_level_div:
                # Attempt to find the 'Hospital-based' span
                hospital_based_span = practice_level_div.find_next('span', string="Small practice")
                if hospital_based_span:
                    # Attempt to find the corresponding 'div class="detail-field"'
                    detail_field = hospital_based_span.find_next('div', class_="detail-field")
                    if detail_field:
                        pl_small_practice_status = detail_field.text.strip()
                    else:
                        pass
                        #pl_hospital_based_status = "Detail field not found for 'Hospital-based'."
                else:
                    pass
                    #pl_hospital_based_status = "'Hospital-based' span not found."
            else:
                pass
                #pl_hospital_based_status = "'Practice Level' div not found."
        else:
            pl_small_practice_status = "'Other Reporting Factors' section not found."
        #print("pl_small_practice_status:", pl_small_practice_status)


        ### Extract (Practice Level) SPECIAL STATUS Health Professional Shortage Area (HPSA)       
        # Attempt to find the section with 'Other Reporting Factors'
        section = soup.find('h6', string="Other Reporting Factors").parent if soup.find('h6', string="Other Reporting Factors") else None       
        if section:
            # Attempt to find the 'Practice Level' div
            practice_level_div = section.find('div', class_="h7", string="Practice Level")
            if practice_level_div:
                # Attempt to find the 'Hospital-based' span
                hospital_based_span = practice_level_div.find_next('span', string="Health Professional Shortage Area (HPSA)")
                if hospital_based_span:
                    # Attempt to find the corresponding 'div class="detail-field"'
                    detail_field = hospital_based_span.find_next('div', class_="detail-field")
                    if detail_field:
                        pl_hpsa_status = detail_field.text.strip()
                    else:
                        pass
                        #pl_hospital_based_status = "Detail field not found for 'Hospital-based'."
                else:
                    pass
                    #pl_hospital_based_status = "'Hospital-based' span not found."
            else:
                pass
                #pl_hospital_based_status = "'Practice Level' div not found."
        else:
            pl_hpsa_status = "'Other Reporting Factors' section not found."
        #print("pl_hpsa_status:", pl_hpsa_status)


        

        return [npi, data_fetch_status, year, associated_practices, physician_name, practice_name, practice_address, individual_status, group_status, optin_text, mips_reporting_requirements, mips_reporting_options, payment_information, cl_exceeds_low_volume, cl_med_patients, cl_allowed_charges, cl_covered_services, cl_mips_ec_type, cl_enroll_b4_2024, pl_exceed_lv, pl_medicare_pts, pl_allowed_charges, pl_covered_services, apm_participation_num, apm_participation_text, apm_participation_comment, apm_name, apm_details_classification, apm_details_model, apm_details_participation_details, cl_hospital_based_status, cl_npfacing_status, cl_rural_status, cl_small_practice_status, cl_hspa_status, cl_hardship, pl_hospital_based_status, pl_rural_status, pl_small_practice_status, pl_hpsa_status]
        #return [npi, associated_practices, physician_name, practice_name, practice_address, mips_info]

    except Exception as e:
        print(f"Error fetching data for NPI {npi}: {str(e)}")
        #return [npi, 'Error', 'Error', 'Error', 'Error']
        return [npi, str(e)]


# Set up the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

## These are the variables to tune the way the program runs
##
##
max_fetch_attempts = 5  # Sometimes the qpp page returns that 2024 is not a supported year. 
                        # This appears to be a glitch in their system because when you check
                        # it by plugging the NPI in by hand, you get good data. 
                        # This variable sets how many times you want to retry an HTML pull for that NPI
                        # before giving up. If it fails to read NPI data after max tries times, you'll see a
                        # "Failed: Unsupported performance year after manimum retries" in the output file
                        # for that NPI

# Variable to specify the input filename for the list of the NPI numbers to read. 
#input_filename = 'Some Other Filename That You Have.txt'  # You can set this variable to any filename as required
input_filename = 'NPI-4.txt'  # You can set this variable to any filename as required

## The QPP Site has data for years 2022, 2023, and 2024. For each NPI in input_filename, the program will read 
## each of the years specified in years.
years = [2022, 2023, 2024]  # List of years
#years = [2024]  # Uncomment this one to extract data from 2024 only

# Read NPI numbers from file
#with open('NPI.txt', 'r') as file:
with open(input_filename, 'r') as file:
    print("Opening the NPI text file")
    npi_numbers = [line.strip() for line in file]
    # Using list comprehension to filter out empty strings
    npi_numbers = [npi for npi in npi_numbers if npi]
    #print("npi_numbers = ", npi_numbers)
    #print()
    cleaned_npi_numbers = []  # List to hold cleaned NPI numbers
    for npi in npi_numbers:
        if not npi.isascii():  # Check if the NPI contains non-ASCII characters
            original_npi = npi  # Store the original NPI for reference
            cleaned_npi = re.sub(r'[^\x00-\x7F]+', '', npi)  # Remove non-ASCII characters using regex
            cleaned_npi_numbers.append(cleaned_npi)
            print(f"Non-ASCII characters were found in NPI {original_npi}. Cleaned NPI: {cleaned_npi}")
        else:
            cleaned_npi_numbers.append(npi)  # If no non-ASCII characters, add the original NPI to the list

    # Calculate the number of elements in the list
    number_of_npi = len(cleaned_npi_numbers)
    print("Number of NPI numbers:", number_of_npi)
    print()

# Prepare CSV output
    #print("Opening the output .csv file")

    # Get the current datetime formatted as YYYYMMDDhhmmss
current_time = datetime.now().strftime('%Y%m%d%H%M%S')
# Dynamically generate the output filename based on the input filename
output_filename = f'{current_time} {input_filename.replace(".txt", "")} REPORT.csv'  
years = [2022, 2023, 2024]  # List of years
#years = [2024]  # List of years
number_of_years = len(years)
number_of_npi_counting_years = number_of_npi*number_of_years
remaining_npi = number_of_npi*number_of_years # seed remaining_npi
processed_npi = 0 # seed the number of npi processed to 0
#print("number_of_npi_counting_years:", number_of_npi_counting_years)


#with open('MIPS REPORTING PER NPI.csv', 'w', newline='', encoding='utf-8') as csvfile:
with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:

    writer = csv.writer(csvfile)
    writer.writerow(['NPI', 'Data Fetch Status', 'Year','Associated Practices', 'Provider Name', 'Practice Name', 'Practice Address', 'MIPS Eligibility: Individual', 'MIPS Eligibility: Group', 'Opt-in Option', 'MIPS Reporting Requirements', 'MIPS Reporting & Participating Options', 'Payment Information', '(Clinician Level) Exceeds low volume threshold', '(Clinician Level) Medicare patients for this clinician', '(Clinician Level) Allowed charges for this clinician', '(Clinician Level) Covered services for this clinician', '(Clinician Level) MIPS eligible clinician type', '(Clinician Level) Enrolled in Medicare before January 1, 2024', '(Practice Level) Exceeds low volume threshold', '(Practice Level) Medicare patients at this practice', '(Practice Level) Allowed charges at this practice', '(Practice Level) Covered services at this practice', 'APM Participation', 'APM Participation Requirements', 'APM Participation Requirements Comment', 'APM Name', '(APM Details) Classification', '(APM Details) Model', '(APM Details) Participation Details', '(Clinician Level) SPECIAL STATUS Hospital-based', '(Clinician Level) SPECIAL STATUS Non-patient facing', '(Clinician Level) SPECIAL STATUS Rural', '(Clinician Level) SPECIAL STATUS Small Practice', '(Clinician Level) Health Professional Shortage Area (HPSA)', '(Clinician Level) (HARDSHIP EXCEPTION)','(Practice Level) SPECIAL STATUS Hospital-based', '(Practice Level) SPECIAL STATUS Rural', '(Practice Level) SPECIAL STATUS Small Practice', '(Practice Level) Health Professional Shortage Area (HPSA)'])
 
    total_time = 0
    # Fetch and write data for each NPI
    for i, npi in enumerate(cleaned_npi_numbers, 1):  # Using enumerate to track index starting from 1
 
        for year in years:
            #max_fetch_attempts = 3
            fetch_attempt = 0
            data = []
                   
            while fetch_attempt < max_fetch_attempts:
                start_time = time.time()
                print(f"Beginning HTML scrape for NPI {npi} for the year {year}")
                data = fetch_info(npi, year) 
                #print("data: ", data)
                print()
                print()
                
                # Check if data contains the unsupported year message
                error_message = "is not a supported performance year"
                error_found = any(error_message in str(item) for item in data)  # Convert items to string to safely use 'in'
                if error_found:
                    print("Error message found!")
                    print(f"Attempt {fetch_attempt + 1}: Unsupported performance year found. Retrying...")
                    fetch_attempt += 1
                    if fetch_attempt == max_fetch_attempts:
                        print(f"Maximum attempts reached for NPI {npi} for the year {year}. Proceeding with the next.")
                        data = [npi, "Failed: Unsupported performance year after maximum retries", year]
                        break
                    continue  # Continue to retry
                else:
                    # If no error related to unsupported year, exit the loop
                    print("Did not detect Not a Supported Performance Year")
                    print()
                    break
                                       
                      
            print("Data to write to csv =", data)
            writer.writerow(data)
            #print("Scrape complete for NPI", npi)
            print("Scrape complete for NPI", npi, "for the year", year)
  
            duration = time.time() - start_time
            total_time += duration
       
            # Calculate average time per NPI and estimate remaining time
            processed_npi = processed_npi + 1 # incrememnt the number of NPI processed
            remaining_npi = number_of_npi_counting_years - processed_npi

            print("processed_npi: ", processed_npi)
            print("remaining_npi: ", remaining_npi)
 
            #average_time = total_time / i
            average_time = total_time / processed_npi

            estimated_remaining_time = average_time * remaining_npi
            # Format remaining time in minutes
            estimated_remaining_time_minutes = estimated_remaining_time / 60

            # Store calculations in variables before printing
            #processed = i
            #remaining = number_of_npi - i
            average_time_formatted = f"{average_time:.2f}"
            estimated_remaining_time_formatted = f"{estimated_remaining_time_minutes:.2f}"

            # Print the number of NPI numbers processed and remaining
            print(f"Processed: {processed_npi}/{number_of_npi_counting_years}, Remaining: {remaining_npi}")
            print(f"Average processing time per NPI: {average_time_formatted} seconds")
            print(f"Estimated remaining time: {estimated_remaining_time_formatted} minutes")
            print()        


# Close the Selenium WebDriver
driver.quit()
print("Program Complete")

The chromedriver version (124.0.6367.91) detected in PATH at C:\Windows\chromedriver.exe might not be compatible with the detected chrome version (125.0.6422.76); currently, chromedriver 125.0.6422.78 is recommended for chrome 125.*, so it is advised to delete the driver in PATH and retry


Opening the NPI text file
Number of NPI numbers: 4

Beginning HTML scrape for NPI 1306927249 for the year 2022
Year:  2022
Page has NOT stabilized.
Page has NOT stabilized.
Page has stabilized.
Number of Associated Practices: 1


Did not detect Not a Supported Performance Year

Data to write to csv = ['1306927249', 'Success', 2022, 1, 'MARK GERALD BROOKS', 'MARK G BROOKS M D P A', '10000 W COLONIAL DRIVE SUITE 187, OCOEE, FL 34761-3498', 'MIPS Exempt Individual', 'MIPS Exempt Group', 'None', 'This clinician is not required to report because they are a Qualifying APM Participant (QP)', 'This clinician may voluntarily report as an individual or group, and receive performance feedback.', 'This clinician is eligible to receive a 5% APM Incentive Payment. If they voluntarily report this will not impact their 5% APM Incentive payment.', 'Yes', 'Exceeds 200', 'Exceeds $90,000', 'Exceeds 200', 'Yes', '', 'Yes', 'Exceeds 200', 'Exceeds $90,000', 'Exceeds 200', '1', 'Check APM Requirements', 'MA

Page has NOT stabilized.
Page has stabilized.
Specific error message found: 2023 is not a supported performance year
filename:  errors\1720084114.html
HTML content has been written to errors\1720084114.html for further inspection.


Error message found!
Attempt 1: Unsupported performance year found. Retrying...
Beginning HTML scrape for NPI 1720084114 for the year 2023
Year:  2023
Page has NOT stabilized.
Page has stabilized.
Number of Associated Practices: 1


Did not detect Not a Supported Performance Year

Data to write to csv = ['1720084114', 'Success', 2023, 1, 'MICHAEL L MAWBY', 'INFUSION ASSOCIATES PLLC', '44720 HAYES RD STE 200, CLINTON TOWNSHIP, MI 48038-1091', 'MIPS Eligible Individual', 'MIPS Eligible Group', 'None', "This clinician is required to report because they're a MIPS eligible clinician type, enrolled in Medicare before the performance year, and exceed the individual low-volume threshold.", 'This clinician is a MIPS APM participant, and can report the APM Performanc

Program Complete
